# Lesson 5.2: Basic Memory Types

---

In Lesson 5.1, we learned about the importance of **Memory** in helping Large Language Models (LLMs) maintain context in conversations. This lesson will focus on two of the most basic and commonly used Memory types in LangChain: **ConversationBufferMemory** and **ConversationBufferWindowMemory**.

## 1. ConversationBufferMemory

### 1.1. Concept and How it Works

**ConversationBufferMemory** is the simplest type of Memory. It stores the **entire conversation history** (both user input and AI output) as a string or a list of message objects.

* **Storage:** Each time there's a new conversation turn (user asks, AI responds), ConversationBufferMemory adds that (input, output) pair to its buffer.
* **Retrieval:** When the LLM needs context, the entire content of the buffer is passed into the prompt.



### 1.2. Pros and Cons

* **Pros:**
    * **Simplicity:** Very easy to set up and use.
    * **Full Context Retention:** Ensures no information is lost from previous conversation turns.
    * **Suitable for Short Conversations:** Effective for interactions that are not too long.

* **Cons:**
    * **Risk of Memory/Token Overflow:** If the conversation becomes too long, the size of the conversation history can exceed the LLM's token limits, leading to errors or truncation of information.
    * **Increased Cost:** Passing a large number of tokens in each LLM call will increase API costs.
    * **Reduced Performance:** The LLM has to process a large amount of potentially unnecessary text, which can slow down response times.

### 1.3. Practical Example Using ConversationBufferMemory with ConversationChain

We will use `ConversationBufferMemory` with `ConversationChain` to see how it maintains context.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# 2. Initialize ConversationBufferMemory
# By default, it will store history as a string in the 'history' variable
memory = ConversationBufferMemory()

# 3. Define Prompt for ConversationChain
# This prompt will have a placeholder for conversation history.
# ConversationChain will automatically populate this placeholder.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer user questions. Maintain conversation context."),
    MessagesPlaceholder(variable_name="history"), # Placeholder for conversation history
    ("human", "{input}"),
])

# 4. Initialize ConversationChain with Memory
# verbose=True to see the chain's operation, including the history passed to the LLM
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt, # Pass custom prompt
    verbose=True
)

print("--- Practical Example with ConversationBufferMemory ---")

# Conversation turn 1
user_input_1 = "What is the capital of France?"
print(f"\nYou: {user_input_1}")
response_1 = conversation.invoke({"input": user_input_1})
print(f"AI: {response_1['response']}")

# Conversation turn 2 (LLM remembers context)
user_input_2 = "What is its population?"
print(f"\nYou: {user_input_2}")
response_2 = conversation.invoke({"input": user_input_2})
print(f"AI: {response_2['response']}")

# Conversation turn 3 (LLM still remembers context)
user_input_3 = "What are its famous tourist attractions?"
print(f"\nYou: {user_input_3}")
response_3 = conversation.invoke({"input": user_input_3})
print(f"AI: {response_3['response']}")

print("\n--- Full History Stored in Memory ---")
print(memory.buffer) # View the entire stored history

## 2. ConversationBufferWindowMemory

### 2.1. Concept and How it Works

**ConversationBufferWindowMemory** is an improvement over ConversationBufferMemory. Instead of storing the entire history, it only stores a **window** of the last `k` messages (or `k` input/output pairs). When a new message is added, the oldest message outside the window is discarded.

* **`k`**: The most important parameter, defining the size of the window.
* **Storage:** Only keeps the last `k` messages.
* **Retrieval:** Only these `k` messages are passed into the LLM's prompt.



### 2.2. Pros and Cons

* **Pros:**
    * **Memory Size Control:** Prevents conversation history from becoming too large, helping to avoid exceeding LLM token limits.
    * **Token Cost Control:** Reduces the number of tokens passed in each LLM call, saving costs.
    * **Suitable for Short-Term Context:** Good for conversations where only recent turns are important.

* **Cons:**
    * **Loss of Long-Term Context:** Important information from conversation turns older than `k` messages will be lost, potentially causing the LLM to forget crucial details.
    * **Careful `k` Adjustment Needed:** Choosing the right `k` value is important to balance context retention and size control.

### 2.3. Practical Example Using ConversationBufferWindowMemory

We will practice with `ConversationBufferWindowMemory` to see how it only retains a portion of the history.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# 2. Initialize ConversationBufferWindowMemory with k=2
# This means it will only remember the last 2 message pairs (input + output)
window_memory = ConversationBufferWindowMemory(k=2)

# 3. Define Prompt for ConversationChain (similar to above)
prompt_window = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý hữu ích. Hãy trả lời các câu hỏi của người dùng. Duy trì ngữ cảnh cuộc trò chuyện."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# 4. Initialize ConversationChain with Window Memory
conversation_window = ConversationChain(
    llm=llm,
    memory=window_memory,
    prompt=prompt_window,
    verbose=True
)

print("\n--- Practical Example with ConversationBufferWindowMemory (k=2) ---")

# Conversation turn 1
user_input_w1 = "Tôi tên là An."
print(f"\nBạn: {user_input_w1}")
response_w1 = conversation_window.invoke({"input": user_input_w1})
print(f"AI: {response_w1['response']}")

# Conversation turn 2
user_input_w2 = "Bạn có thể giúp tôi làm gì?"
print(f"\nBạn: {user_input_w2}")
response_w2 = conversation_window.invoke({"input": user_input_w2})
print(f"AI: {response_w2['response']}")

# Conversation turn 3 (An's first message will be discarded from the window)
user_input_w3 = "Tôi muốn hỏi về trí tuệ nhân tạo."
print(f"\nBạn: {user_input_w3}")
response_w3 = conversation_window.invoke({"input": user_input_w3})
print(f"AI: {response_w3['response']}")

# Conversation turn 4 (An's second message will be discarded from the window)
user_input_w4 = "Trí tuệ nhân tạo là gì?"
print(f"\nBạn: {user_input_w4}")
response_w4 = conversation_window.invoke({"input": user_input_w4})
print(f"AI: {response_w4['response']}")

print("\n--- History in Window Memory (k=2) ---")
print(window_memory.buffer) # View the stored history (only the last 2 message pairs)

# Try asking for the name again (LLM might not remember if it was pushed out of the window)
user_input_w5 = "Tên tôi là gì?"
print(f"\nBạn: {user_input_w5}")
response_w5 = conversation_window.invoke({"input": user_input_w5})
print(f"AI: {response_w5['response']}")

**Explanation:**
* With `k=2`, `ConversationBufferWindowMemory` only keeps the last 2 message pairs.
* You will see in the `verbose=True` output that when `user_input_w3` is added, the first message pair ("Tôi tên là An." and AI's response) has been removed from the history sent to the LLM.
* Therefore, when you ask "Tên tôi là gì?" in turn `user_input_w5`, the LLM might not remember your name because that information has been pushed out of the memory window.


---

## Tóm tắt Bài học

Bài học này đã đi sâu vào hai loại **Memory** cơ bản trong LangChain: **ConversationBufferMemory** và **ConversationBufferWindowMemory**. Bạn đã hiểu rằng **ConversationBufferMemory** lưu trữ toàn bộ lịch sử cuộc trò chuyện, đơn giản nhưng có thể gặp vấn đề về giới hạn token cho các cuộc hội thoại dài. Ngược lại, **ConversationBufferWindowMemory** chỉ lưu trữ một cửa sổ gồm `k` tin nhắn gần nhất, giúp kiểm soát kích thước bộ nhớ và chi phí token, nhưng có thể làm mất ngữ cảnh dài hạn. Thông qua các ví dụ thực hành với `ConversationChain`, bạn đã thấy cách mỗi loại Memory hoạt động và cách chúng ảnh hưởng đến khả năng duy trì ngữ cảnh của LLM.